# Data Arrangement

Because the medicare data from cms is poorly labelled, it is hard to write a script to download it. But supposing you have downloaded penetration data and the county-state-plan subscription data, here is a workbook that unzips all the files and makes them into a single panel data set, and saves it in Stata form. So:

In [1]:
import os, zipfile, glob, re

import pandas as pd
import numpy as np

In [2]:
dir_name_1 = 'C:\\Users\\matthew\\Documents\\Medicare\\Markets\\'
dir_name_2 = 'C:\\Users\\matthew\\Documents\\Medicare\\Shares\\'
extension = ".zip"

In [3]:
os.chdir(dir_name_1) # change directory from working dir to dir with files

for item in os.listdir(dir_name_1):            # loop through items in dir
    if item.endswith(extension):               # check for ".zip" extension
        file_name = os.path.abspath(item)      # get full path of files
        zip_ref = zipfile.ZipFile(file_name)   # create zipfile object
        zip_ref.extractall(dir_name_1)         # extract file to dir
        zip_ref.close()                        # close file

In [4]:
os.chdir(dir_name_2)

for item in os.listdir(dir_name_2):
    if item.endswith(extension):
        file_name = os.path.abspath(item)
        zip_ref = zipfile.ZipFile(file_name)
        zip_ref.extractall(dir_name_2)
        zip_ref.close()

In [5]:
current_dir = os.getcwd()

os.chdir(dir_name_1)

file_list_1 = []
for file in glob.glob("*.csv"):
    file_list_1.append(file)

In [6]:
os.chdir(dir_name_2)

file_list_2 = []
for file in glob.glob("*.csv"):
    file_list_2.append(file)

# Making DataFrames

In [7]:
pendata = pd.read_csv(dir_name_1 + file_list_1[0])
pendata['fn'] = file_list_1[0]

for file in file_list_1:
    df = pd.read_csv(dir_name_1 + file)
    df['fn'] = file
    pendata = pendata.append(df)

# Read the Pickle...if previously made

In [8]:
try:
    shrdata = pd.read_pickle('C:\\Users\\matthew\\downloads\\tempshr.pkl')
    print('File exists and is read in...')
except:
    shrdata = pd.read_csv(dir_name_2 + file_list_2[0])
    shrdata['fn'] = file_list_2[0]

    for file in file_list_2:
        print(len(shrdata), end='.')
        df = pd.read_csv(dir_name_2 + file)
        df['fn'] = file
        shrdata = shrdata.append(df)
    
    shrdata.to_pickle('C:\\Users\\matthew\\downloads\\tempshr.pkl')

File exists and is read in...


# Taking a look at the data

We first see that our data sets are quite long! Unfortunately, we need to make them a little longer by adding in information about the year and the month. Let's first see if we don't need some of the observations.

In [27]:
shrdata.head()

,County,State,Contract ID,Organization Name,Organization Type,Plan Type,SSA Code,FIPS Code,Enrolled_x,year,...,FIPSST,FIPSCNTY,FIPS,SSAST,SSACNTY,SSA,Eligibles,Enrolled_y,Penetration,_merge
0,Autauga,AL,H0104,BLUE CROSS AND BLUE SHIELD OF ALABAMA,Local CCP,Local PPO,1000.0,1001.0,593.0,2008.0,...,01,1.0,01001,1.0,0.0,1000.0,"7,494","2,003",26.73%,both
1,Autauga,AL,H0104,BLUE CROSS AND BLUE SHIELD OF ALABAMA,Local CCP,Local PPO,1000.0,1001.0,593.0,2008.0,...,01,1.0,01001,1.0,0.0,1000.0,"7,494","2,003",26.73%,both
2,Autauga,AL,H0150,"HEALTHSPRING OF ALABAMA, INC.",Local CCP,HMO/HMOPOS,1000.0,1001.0,218.0,2008.0,...,01,1.0,01001,1.0,0.0,1000.0,"7,494","2,003",26.73%,both
3,Autauga,AL,H0150,"HEALTHSPRING OF ALABAMA, INC.",Local CCP,HMO/HMOPOS,1000.0,1001.0,218.0,2008.0,...,01,1.0,01001,1.0,0.0,1000.0,"7,494","2,003",26.73%,both
4,Autauga,AL,H0154,"VIVA HEALTH, INC.",Local CCP,HMO/HMOPOS,1000.0,1001.0,874.0,2008.0,...,01,1.0,01001,1.0,0.0,1000.0,"7,494","2,003",26.73%,both


We will use the file names that we affixed to get a year and a date for each observation as we need this to merge on. In any event:

In [10]:
years  = [re.findall(r"\d{4}", file)[0] for file in file_list_2]
years  = [float(item) for item in years]

months = [re.findall(r"\_\d{2}\.", file)[0][1:3] for file in file_list_2]
months = [float(item) for item in months]

In [11]:
shr_data_year_dict = dict(zip(file_list_2, years))
shr_data_mont_dict = dict(zip(file_list_2, months))

In [12]:
shrdata['year'] = shrdata['fn'].map(shr_data_year_dict)

In [13]:
shrdata['month'] = shrdata['fn'].map(shr_data_mont_dict)

In [14]:
shrdata.drop(['fn'], axis=1, inplace=True)

In [15]:
years = [re.findall(r"\d{4}", file)[0] for file in file_list_1]
years = [float(item) for item in years]

months = [re.findall(r"\_\d{2}\.", file)[0][1:3] for file in file_list_1]
months = [float(item) for item in months]


In [16]:
pen_data_year_dict = dict(zip(file_list_1, years))
pen_data_mont_dict = dict(zip(file_list_1, months))

pendata['year']  = pendata['fn'].map(pen_data_year_dict)
pendata['month'] = pendata['fn'].map(pen_data_mont_dict) 

pendata.drop(['fn'], axis=1, inplace=True)

# Merging Data

Before, we merged on state, county, and other stuff. I guess we could try this on the codes and all that. The problem is that there are FIPS codes that mix floats and strings. I don't trust them so I will just do the following:

In [17]:
state_match = {'Alabama':'AL', 'Alaska':'AK', 'American Samoa':'AS', 'Arizona':'AZ', 'Arkansas':'AR', 'California':'CA',
 'Colorado':'CO', 'Connecticut':'CT', 'Delaware':'DE', 'District Of Columbia':'DC',
 'Florida':'FL', 'Georgia':'GA', 'Guam':'GU', 'Hawaii':'HI', 'Idaho':'ID', 'Illinois':'IL',
 'Indiana':'IN', 'Iowa':'IA', 'Kansas':'KS', 'Kentucky':'KY', 'Louisiana':'LA', 'Maine':'ME', 'Maryland':'MD', 
 'Massachusetts':'MA',
 'Michigan':'MI', 'Minnesota':'MN', 'Mississippi':'MS', 'Missouri':'MO', 'Montana':'MT', 'Nebraska':'NE', 'Nevada':'NV', 
 'New Hampshire':'NH', 'New Jersey':'NJ', 'New Mexico':'NM',
 'New York':'NY', 'North Carolina':'NC', 'North Dakota':'ND', 'Ohio':'OH',
 'Oklahoma':'OK', 'Oregon':'OR', 'Pending State Designation':'GB', 'Pennsylvania':'PA', 'Puerto Rico':'PR', 
 'Rhode Island':'RI', 'South Carolina':'SC',
 'South Dakota':'SD', 'Tennessee':'TN', 'Texas':'TX', 'Utah':'UT', 'Vermont':'VT', 'Virgin Islands':'VI',
 'Virginia':'VA',
 'Wake Island':'QW', 'Washington':'WA', 'Washington D.C.':'DC', 'West Virginia':'WV', 'Wisconsin':'WI', 'Wyoming':'WY'}

In [18]:
pendata['State'] = pendata['State Name'].map(state_match)

In [19]:
pendata.rename(columns={'County Name': 'County'}, inplace=True)

We now should be able to merge on County, State, year and month...which will probably take a while! However, there seems to be just too much data to actually pull this off. So, let's try and drop nan variables. First, what do we gain by doing this? 

In [20]:
shrdata['Enrolled'].notna().sum()

3150353

In [21]:
shrdata = shrdata.loc[shrdata['Enrolled'].notna()]

In [22]:
shrdata = pd.merge(shrdata, pendata, on=['State', 'County', 'year', 'month'], how='outer', indicator=True)

In [23]:
ro = shrdata[['County', 'State', 'year', 'month']].loc[shrdata['_merge'] == 'right_only']
lo = shrdata[['County', 'State', 'year', 'month']].loc[shrdata['_merge'] == 'left_only']

In [24]:
stateco = ro['County'] + ' ' + ro['State']
stateco2 = lo['County'] + ' ' + lo['State']

From the looks of the above, it seems as though most of the data from the left is out of the united states proper. It could be that no medicare advantage plans are offered in the rest of the places. 

In [25]:
obj_cols = list(shrdata.select_dtypes(include=['object']).columns)

for col in obj_cols:
    shrdata[col] = shrdata[col].astype(str)

In [26]:
shrdata.to_stata('C:\\Users\\matthew\\downloads\\medicare.dta')

C:\Users\Matthew\Anaconda3\lib\site-packages\pandas\io\stata.py:2138: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    b'Contract ID'   ->   Contract_ID
    b'Organization Name'   ->   Organization_Name
    b'Organization Type'   ->   Organization_Type
    b'Plan Type'   ->   Plan_Type
    b'SSA Code'   ->   SSA_Code
    b'FIPS Code'   ->   FIPS_Code
    b'State Name'   ->   State_Name

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  warnings.warn(ws, InvalidColumnName)
